## Подзапросы, представления, создание таблиц

**1.** Данные для следующих задач: Airbnb в Берлине.

Сначала оставьте только те объявления, в которых оценка на основе отзывов выше среднего, а число отзывов в месяц составляет строго меньше трёх. Затем отсортируйте по убыванию две колонки: сначала по числу отзывов в месяц, потом по оценке. В качестве ответа укажите id объявления из первой строки.

* **review_scores_rating** – оценка на основе отзывов
* **reviews_per_month** – число отзывов в месяц
* **id** – идентификатор объявления

Таблица – **listings**. Наивысшие показатели = сортировка по убыванию.

<div class="alert alert-block alert-warning"
<b>
<ol>
<b><li>SELECT</li></b> 
<b><li>    id,</li></b>
<b><li>    reviews_per_month, </li></b>
<b><li>    toFloat64OrNull(review_scores_rating) as review_scores_rating </li></b>
<b><li>FROM listings</li></b>
<b><li>WHERE </li></b>
<b><li>    review_scores_rating > (SELECT AVG(toFloat64OrNull(review_scores_rating))</li></b>
<b><li>                FROM listings)</li></b>
<b><li>AND reviews_per_month < 3</li></b>
<b><li>ORDER BY </li></b>
<b><li>    review_scores_rating DESC,</li></b>
<b><li>    reviews_per_month DESC</li></b>
<b><li>LIMIT 1</li></b>
</ol>
</b>
</div>

**2.** Посчитайте среднее расстояние до центра города и выведите идентификаторы объявлений о сдаче отдельных комнат, для которых расстояние оказалось меньше среднего. Результат отсортируйте по убыванию, тем самым выбрав комнату, которая является наиболее удаленной от центра, но при этом расположена ближе, чем остальные комнаты в среднем.

* **id** – идентификатор объявления
* **host_id** – идентификатор хозяина
* **room_type** – тип жилья ('Private room')
* **latitude** – широта
* **longitude** – долгота
* **52.5200 с.ш., 13.4050 в.д** – координаты центра Берлина

В качестве ответа укажите идентификатор хозяина (host_id), сдающего данную комнату.

* **geoDistance(13.4050, 52.5200, долгота, широта)**

**Note:** задание можно решить как используя подзапрос в **WHERE**, так и с помощью **WITH**. При подсчете среднего тип комнаты тоже следует учесть.

<div class="alert alert-block alert-warning"
<b>
<ol>
<b><li>SELECT </li></b>
<b><li>    id,</li></b>
<b><li>    host_id,</li></b>
<b><li>    geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) as distance</li></b>
<b><li>FROM listings</li></b>
<b><li>WHERE </li></b>
<b><li>    distance < (SELECT AVG(geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)))</li></b>
<b><li>                FROM listings</li></b>
<b><li>                WHERE room_type == 'Private room')</li></b>
<b><li>AND room_type == 'Private room'</li></b>
<b><li>ORDER BY distance DESC</li></b>
<b><li>LIMIT 1</li></b>
</ol>
</b>
</div>

**3.** Представим, что вы планируете снять жилье в Берлине на 7 дней, используя более хитрые фильтры, чем предлагаются на сайте.

**В этой задаче можно потренироваться в написании подзапросов, но задание можно решить и без них.**

Отберите объявления из таблицы **listings**, которые:

* находятся на расстоянии от центра меньше среднего (здесь нам пригодится запрос из предыдущего задания)
* обойдутся дешевле 100$ в день (**price** с учетом **cleaning_fee**, который добавляется к общей сумме за неделю, т.е его нужно делить на кол-во дней)
* имеют последние отзывы (**last_review**), начиная с 1 сентября 2018 года
* имеют WiFi в списке удобств (**amenities**)

Отсортируйте полученные значения по убыванию **review_scores_rating** (не забудьте перевести строку к численному виду) и в качестве ответа укажите **host_id** из первой строки. 

<div class="alert alert-block alert-warning"
<b>
<ol>
<b><li>SELECT </li></b>
<b><li>    host_id,</li></b>
<b><li>    geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) as distance,</li>

<li>    (toFloat32OrNull(replaceRegexpAll(cleaning_fee, '[$,]', ''))/7) + </li>

<li>    (toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) as full_price,</li>
<li>    last_review,</li>
<li>    multiSearchAnyCaseInsensitive(amenities, ['WiFi']) as consists_Wifi,</li></b>
<b><li>    toFloat32OrNull(review_scores_rating) as review_scores_rating  </li></b>
<b><li>FROM listings</li></b>
<b><li>WHERE </li></b>
<b><li>    distance < (SELECT AVG(geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)))</li></b>
<b><li>                FROM listings</li></b>
<b><li>                WHERE room_type == 'Private room')</li></b>
<b><li>AND room_type == 'Private room'</li></b>
<b><li>AND full_price < 100   </li></b>
<b><li>AND last_review > '2018-09-01'</li></b>
<b><li>AND consists_Wifi == '1'</li></b>
<b><li>ORDER BY review_scores_rating  DESC</li></b>
<b><li>LIMIT 1</li></b>
</ol>
</b>
</div>

**4.** Давайте найдем в таблице **calendar_summary** те доступные (**available='t'**) объявления, у которых число отзывов от уникальных пользователей в таблице **reviews** выше среднего.

**NB!** Для простоты будем считать, что **отзыв** — это уникальный посетитель на уникальное жилье, не учитывая возможные повторные отзывы от того же посетителя.

Для этого с помощью конструкции **WITH** посчитайте среднее число уникальных **reviewer_id** из таблицы **reviews** на каждое жильё, потом проведите джойн таблиц **calendar_summary** и **reviews** по полю **listing_id** (при этом из таблицы **calendar_summary** должны быть отобраны уникальные **listing_id**, отфильтрованные по правилу **available='t'**). Результат отфильтруйте так, чтобы остались только записи, у которых число отзывов от уникальных людей выше среднего.

Отсортируйте результат по возрастанию **listing_id** и в качестве ответа впишите количество отзывов от уникальных пользователей из первой строки.

**Hint:** для решения проблем со вложенными агрегационными функциями (и агрегационными функциями там, где их не ждут) как раз очень помогут подзапросы.

<div class="alert alert-block alert-warning"
<b>
<ol>
<b><li>WITH</li></b>
<b><li>(SELECT AVG(count_rev) FROM (SELECT COUNT(DISTINCT reviewer_id) as count_rev, listing_id FROM reviews GROUP BY listing_id)) <b><li>as avg_rev</li></b>


<b><li>SELECT listing_id, count_rev</li></b>
<b><li>FROM </li></b>
<b><li>    (SELECT DISTINCT listing_id</li></b>
<b><li>    FROM</li></b>
<b><li>    calendar_summary</li></b>
<b><li>    WHERE available='t') as a</li></b>
<b><li>JOIN </li></b>
<b><li>    (SELECT COUNT(DISTINCT reviewer_id) as count_rev, listing_id</li></b>
<b><li>    FROM reviews </li></b>
<b><li>    GROUP BY listing_id) as b</li></b>
<b><li>ON a.listing_id = b.listing_id</li></b>
<b><li>WHERE count_rev > avg_rev</li></b>
<b><li>ORDER BY listing_id ASC</li></b>
<b><li>LIMIT 10</li></b>
</ol>
</b>
</div>

**5.** Возвращаемся к данными мобильного приложения!Используйте таблицу **checks** и разделите всех покупателей на сегменты:

**NB!** Правые границы берутся не включительно, например, чек в 10 рублей будет относиться к сегменту С

* **А** — средний чек покупателя менее 5 ₽
* **B** — средний чек покупателя от 5-10 ₽
* **C** — средний чек покупателя от 10-20 ₽
* **D** — средний чек покупателя от 20 ₽

Отсортируйте результирующую таблицу по возрастанию **UserID** и укажите сегмент **четвертого** пользователя.

<div class="alert alert-block alert-warning"
<b>
<ol>
<b><li>SELECT</li></b>
<b><li>    UserID,</li></b>
<b><li>    CASE</li></b>
<b><li>        WHEN AVG(Rub) < 5 THEN 'A'</li></b>
<b><li>        WHEN AVG(Rub) >= 5 AND AVG(Rub) < 10 THEN 'B'</li></b>
<b><li>        WHEN AVG(Rub) >= 10 AND AVG(Rub) < 20 THEN 'C'</li></b>
<b><li>        ELSE 'D'</li></b>
<b><li>    END AS seg</li></b>
<b><li>FROM checks</li></b>
<b><li>GROUP BY UserID</li></b>
<b><li>ORDER BY UserID ASC</li></b>
<b><li>LIMIT 10</li></b>
</ol>
</b>
</div>

**6.** Используйте предыдущий запрос как подзапрос и посчитайте, сколько клиентов приходится на каждый сегмент и сколько доходов он приносит. Отсортируйте результат по убыванию суммы доходов на сегмент и в качестве ответа укажите наибольшую сумму.

<div class="alert alert-block alert-warning"
<b>
<ol>
<b><li>SELECT</li></b>
<b><li>    seg,</li></b>
<b><li>    COUNT(DISTINCT UserID),</li></b>
<b><li>    SUM(Rub) as Revenue</li></b>
<b><li>FROM</li></b>
<b><li>    checks</li></b>
<b><li>JOIN</li></b>
<b><li>   (SELECT</li></b>
<b><li>    UserID,</li></b>
<b><li>    CASE</li></b>
<b><li>        WHEN avg(Rub) < 5 THEN 'A'</li></b>
<b><li>        WHEN avg(Rub) >= 5 AND avg(Rub) < 10 THEN 'B'</li></b>
<b><li>        WHEN avg(Rub) >= 10 AND avg(Rub) < 20 THEN 'C'</li></b>
<b><li>        ELSE 'D'</li></b>
<b><li>    END AS seg</li></b>
<b><li>    FROM checks</li></b>
<b><li>    GROUP BY UserID) as sub</li></b>
<b><li>USING (UserID)</li></b>
<b><li>GROUP BY seg</li></b>
<b><li>ORDER BY Revenue DESC</li></b>
</ol>
</b>
</div>

Такой же результат можно получить следующим образом:

<div class="alert alert-block alert-warning"
<b>
<ol>
<b><li>SELECT seg, COUNT(UserID) as count_users, SUM(rub_for_user) as revenue</li></b>
<b><li>FROM</li></b>
<b><li>(SELECT</li></b>
<b><li>    UserID, SUM(Rub) as rub_for_user,</li></b>
<b><li>    CASE</li></b>
<b><li>        WHEN AVG(Rub) < 5 THEN 'A'</li></b>
<b><li>        WHEN AVG(Rub) >= 5 AND AVG(Rub) < 10 THEN 'B'</li></b>
<b><li>        WHEN AVG(Rub) >= 10 AND AVG(Rub) < 20 THEN 'C'</li></b>
<b><li>        ELSE 'D'</li></b>
<b><li>    END AS seg</li></b>
<b><li>FROM checks</li></b>
<b><li>GROUP BY UserID</li></b>
<b><li>ORDER BY UserID ASC)</li></b>
<b><li>GROUP BY seg</li></b>
<b><li>ORDER BY revenue DESC</li></b>
</ol>
</b>
</div>

**7.** Вернемся к таблице AirBnb. Предположим, что в выборе жилья нас интересует только два параметра: наличие кухни (**kitchen**) и гибкой системы отмены (**flexible**), причем первый в приоритете.

Создайте с помощью оператора **CASE** колонку с обозначением группы, в которую попадает жилье из таблицы **listings**:

* **'good'**, если в удобствах (**amenities**) присутствует кухня и система отмены (**cancellation_policy**) гибкая
* **'ok'**, если в удобствах есть кухня, но система отмены не гибкая
* **'not ok'** во всех остальных случаях

Результат отсортируйте по новой колонке по возрастанию, установите ограничение в 5 строк, в качестве ответа укажите **host_id** первой строки.

Обратите внимание, что **cancellation_policy** - это отдельная колонка, по ней необходимо смотреть систему отмены

<div class="alert alert-block alert-warning"
<b>
<ol>
<b><li>SELECT host_id,</li></b>
<b><li>    CASE</li> </b>
<b><li>    WHEN multiSearchAnyCaseInsensitive(amenities, ['kitchen'])=1 AND cancellation_policy='flexible' THEN 'good'</li></b>
<b><li>    WHEN multiSearchAnyCaseInsensitive(amenities, ['kitchen'])=1 AND cancellation_policy!='flexible' THEN 'ok'</li></b>
<b><li>    ELSE 'not ok'</li></b>
<b><li>    END as cat</li></b>
<b><li>FROM listings</li></b>
<b><li>ORDER BY cat ASC</li></b>
<b><li>LIMIT 5</li></b>
</ol>
</b>
</div>

**8.** Напишите запрос для создания таблицы со следующими параметрами, также подобрав подходящий тип данных.

**Название таблицы: reviews**

**База данных: test**

**Столбцы:**

- **listing_id** – идентификатор объявления, может быть только положительным и целым числом, 32-битный тип данных

- **id** – идентификатор хозяина, может быть только положительным и целым числом, 32-битный тип данных

- **created_at** – дата со временем (2020-01-01 00:00:00), часовой пояс – 'Europe/Moscow'

- **reviewer_id** – идентификатор ревьюера, может быть только положительным и целым числом, 32-битный тип данных

- **reviewer_name** – имя того, кто оставил отзыв

- **comments** - текст отзыва

**Движок: MergeTree**

**Сортировка: listing_id, id**

<div class="alert alert-block alert-warning"
<b>
<ol>
<b><li>CREATE TABLE test.reviews (</li></b> 
<b><li>    listing_id UInt32, </li></b>
<b><li>    id UInt32,</li></b>
<b><li>    created_at DateTime('Europe/Moscow'),</li></b>
<b><li>    reviewer_id UInt32,</li></b>
<b><li>    reviewer_name String,</li></b>
<b><li>    comments String</li></b>
<b><li>    ) </li></b>
<b><li>ENGINE = MergeTree</li></b>
<b><li>ORDER BY (listing_id, id) </li></b>
</ol>
</b>
</div>

**9.** К вам пришел коллега с новостями: оказывается, в поле **created_at** будет записываться только дата, без времени,
поэтому нужно изменить тип данных!

Напишите запрос, который модифицирует тип данных, и введите его в поле ниже без кавычек и лишних пробелов.
Не забудьте указать базу данных **test** перед названием таблицы! 

<div class="alert alert-block alert-warning"
<b>
<ol>
<b>ALTER TABLE test.reviews MODIFY COLUMN created_at Date</b>
</ol>
</b>
</div>

**10.** Предположим, ваш коллега вставил данные, но что-то перепутал. Часть строк с комментариями осталась совершенно пустой! 

Напишите запрос, который удалит из таблицы **test.reviews** те строки, где в столбце **comments** встречаются пустые значения ('').Введите его в поле ниже без кавычек и лишних пробелов.

<div class="alert alert-block alert-warning"
<b>
<ol>
<b>ALTER TABLE test.reviews DELETE WHERE comments=''</b>
</ol>
</b>
</div>

**11.** С помощью какого запроса можно создать обычное представление над таблицей **test.reviews**, которое будет содержать все записи из **test.reviews**, сгруппированные по **reviewer_id** с подсчитанным количеством отзывов (**id**) на каждого пользователя?

<div class="alert alert-block alert-warning"
<b>
<ol>
<b><li>CREATE VIEW test.reviews_number AS</li></b> 
<b><li>(SELECT reviewer_id, </li></b>
<b><li>        COUNT(id) reviews_count </li></b>
<b><li>FROM test.reviews </li></b>
<b><li>GROUP BY reviewer_id)</li></b>
</ol>
</b>
</div>

**12.** С помощью какого запроса можно создать новый столбец **reviewer_score** в таблице **reviews** после столбца **reviewer_name**?

<div class="alert alert-block alert-warning"
<b>
<ol>
<b>ALTER TABLE test.reviews ADD COLUMN reviewer_score UInt8 AFTER reviewer_name</b>
</ol>
</b>
</div>

**13.** Напишите запрос для добавления в таблицу **test.reviews** колонки **price** после колонки **comments**, которая может быть числом с плавающей точкой, 32-битный тип данных.

<div class="alert alert-block alert-warning"
<b>
<ol>
<b>ALTER TABLE test.reviews ADD COLUMN price Float32 AFTER comments</b>
</ol>
</b>
</div>

**14.** Напишите запрос, который удвоит **price** для всех строк с датой (**created_at**) после 2019-01-01.

Обратите внимание, что для сравнения с датой в формате '2019-01-01' можно не преобразовывать данные в колонке **created_at**.

<div class="alert alert-block alert-warning"
<b>
<ol>
<b>ALTER TABLE test.reviews UPDATE price=price*2 WHERE created_at>'2019-01-01'</b>  
</ol>
</b>
</div>